- https://www.datamanim.com/dataset/03_dataq/typetwo.html#id15

In [ ]:
!pip install xgboost

# 4. 학생성적 예측 데이터(Regression)

> Attention
- 데이터 설명 : 학생성적 예측 (종속변수 :G3)
- 데이터 출처 : https://www.kaggle.com/datasets/ishandutta/student-performance-data-set (참고, 데이터 수정)
- X_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv
- y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv
- X_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv
- y_test(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv

#### 0. 시험 환경 세팅

In [ ]:
import pandas as pd

X_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv")

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((678, 33), (678, 2), (366, 33), (366, 2))

#### 1. 라이브러리 및 데이터 호출

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
X_train.drop('StudentID', axis=1, inplace=True)
X_test.drop('StudentID', axis=1, inplace=True)

y = y_train['G3']
y_test = y_test['G3']

X_train.shape, X_test.shape, y.shape, y_test

((678, 32),
 (366, 32),
 (678,),
 0      16
 1      11
 2      14
 3       6
 4      11
        ..
 361    15
 362    11
 363    17
 364     8
 365    14
 Name: G3, Length: 366, dtype: int64)

#### 2. EDA

In [ ]:
X_train.info(), X_train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      678 non-null    object
 1   sex         678 non-null    object
 2   age         678 non-null    int64 
 3   address     678 non-null    object
 4   famsize     678 non-null    object
 5   Pstatus     678 non-null    object
 6   Medu        678 non-null    int64 
 7   Fedu        678 non-null    int64 
 8   Mjob        678 non-null    object
 9   Fjob        678 non-null    object
 10  reason      678 non-null    object
 11  guardian    678 non-null    object
 12  traveltime  678 non-null    int64 
 13  studytime   678 non-null    int64 
 14  failures    678 non-null    int64 
 15  schoolsup   678 non-null    object
 16  famsup      678 non-null    object
 17  paid        678 non-null    object
 18  activities  678 non-null    object
 19  nursery     678 non-null    object
 20  higher    

(None,
 school        0
 sex           0
 age           0
 address       0
 famsize       0
 Pstatus       0
 Medu          0
 Fedu          0
 Mjob          0
 Fjob          0
 reason        0
 guardian      0
 traveltime    0
 studytime     0
 failures      0
 schoolsup     0
 famsup        0
 paid          0
 activities    0
 nursery       0
 higher        0
 internet      0
 romantic      0
 famrel        0
 freetime      0
 goout         0
 Dalc          0
 Walc          0
 health        0
 absences      0
 G1            0
 G2            0
 dtype: int64)

In [ ]:
X_test.info(), X_test.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      366 non-null    object
 1   sex         366 non-null    object
 2   age         366 non-null    int64 
 3   address     366 non-null    object
 4   famsize     366 non-null    object
 5   Pstatus     366 non-null    object
 6   Medu        366 non-null    int64 
 7   Fedu        366 non-null    int64 
 8   Mjob        366 non-null    object
 9   Fjob        366 non-null    object
 10  reason      366 non-null    object
 11  guardian    366 non-null    object
 12  traveltime  366 non-null    int64 
 13  studytime   366 non-null    int64 
 14  failures    366 non-null    int64 
 15  schoolsup   366 non-null    object
 16  famsup      366 non-null    object
 17  paid        366 non-null    object
 18  activities  366 non-null    object
 19  nursery     366 non-null    object
 20  higher    

(None,
 school        0
 sex           0
 age           0
 address       0
 famsize       0
 Pstatus       0
 Medu          0
 Fedu          0
 Mjob          0
 Fjob          0
 reason        0
 guardian      0
 traveltime    0
 studytime     0
 failures      0
 schoolsup     0
 famsup        0
 paid          0
 activities    0
 nursery       0
 higher        0
 internet      0
 romantic      0
 famrel        0
 freetime      0
 goout         0
 Dalc          0
 Walc          0
 health        0
 absences      0
 G1            0
 G2            0
 dtype: int64)

In [ ]:
int_col = X_train.select_dtypes(exclude='object').columns

print('int_col','\n')

for c in int_col:
    print(c, X_train[c].nunique())
    print(np.sort(X_train[c].unique()))
    print('='*100)

int_col 

age 8
[15 16 17 18 19 20 21 22]
Medu 5
[0 1 2 3 4]
Fedu 5
[0 1 2 3 4]
traveltime 4
[1 2 3 4]
studytime 4
[1 2 3 4]
failures 4
[0 1 2 3]
famrel 5
[1 2 3 4 5]
freetime 5
[1 2 3 4 5]
goout 5
[1 2 3 4 5]
Dalc 5
[1 2 3 4 5]
Walc 5
[1 2 3 4 5]
health 5
[1 2 3 4 5]
absences 31
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 26 30 32 54 56 75]
G1 16
[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
G2 16
[ 0  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [ ]:
obj_col = X_train.select_dtypes(include='object').columns

print('obj col', '\n')

for c in obj_col:
    print(c, X_train[c].nunique())
    print(X_train[c].unique())
    print('='*100)

obj col 

school 2
['GP' 'MS']
sex 2
['F' 'M']
address 2
['U' 'R']
famsize 2
['GT3' 'LE3']
Pstatus 2
['T' 'A']
Mjob 5
['other' 'health' 'services' 'at_home' 'teacher']
Fjob 5
['other' 'teacher' 'services' 'at_home' 'health']
reason 4
['reputation' 'other' 'course' 'home']
guardian 3
['father' 'mother' 'other']
schoolsup 2
['no' 'yes']
famsup 2
['yes' 'no']
paid 2
['yes' 'no']
activities 2
['no' 'yes']
nursery 2
['yes' 'no']
higher 2
['yes' 'no']
internet 2
['yes' 'no']
romantic 2
['no' 'yes']


#### 3. Preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()
X_train[int_col] = scaler.fit_transform(X_train[int_col])
X_test[int_col] = scaler.transform(X_test[int_col])

le = LabelEncoder()
X_train[obj_col] = X_train[obj_col].apply(le.fit_transform)
X_test[obj_col] = X_test[obj_col].apply(le.fit_transform)

#### 4. Modeling

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
rf = RandomForestRegressor(random_state=156)
gb = GradientBoostingRegressor(random_state=156)
xgb = XGBRegressor(random_state=156)
xgbrf = XGBRFRegressor(random_state=156)

models = [rf, gb, xgb, xgbrf]

In [ ]:
params = {'n_estimators':[100, 500], 'max_depth':[1,2,3,4,5]}

In [ ]:
best_estimator_models = []
for model in models:
    gscv = GridSearchCV(model, params, cv=3, scoring='r2', n_jobs = 5)
    gscv.fit(X_train, y_train)
    
    print(str(model).split('(')[0])
    print('params :', gscv.best_params_)
    print('score :', gscv.best_score_)
    best_estimator_models.append(gscv.best_estimator_)
    print('='*10)

RandomForestRegressor
params : {'max_depth': 4, 'n_estimators': 100}
score : 0.8668716120828512
GradientBoostingRegressor
params : {'max_depth': 2, 'n_estimators': 100}
score : 0.8674282407140206
XGBRegressor
params : {'max_depth': 3, 'n_estimators': 100}
score : 0.8537392684812736
XGBRFRegressor
params : {'max_depth': 4, 'n_estimators': 100}
score : 0.8656589219377491


In [ ]:
best_estimator_models[0]

RandomForestRegressor(max_depth=4, random_state=156)

In [ ]:
rf = best_estimator_models[0]

rf.fit(X_train, y)
pred = rf.predict(X_test)

In [ ]:
y_train["G3"]

0      14
1      15
2      11
3      13
4       0
       ..
673    14
674    11
675    13
676    10
677    18
Name: G3, Length: 678, dtype: int64

In [ ]:
out_data = pd.DataFrame({'StudentID':StudentID, 'G3':pred})
out_data

,StudentID,G3
0,1000,16.046516
1,1008,10.917490
2,1013,14.511223
3,1014,7.660486
4,1017,10.792371
...,...,...
361,2032,15.510834
362,2034,12.444338
363,2035,16.057844
364,2036,0.502898


In [ ]:
# 가채점
print('rmse :', np.sqrt(mean_squared_error(y_test, pred)))
print('r2 :', r2_score(y_test, pred))

rmse : 1.6742038706478317
r2 : 0.8177229995730235


In [ ]:
## 딥러닝 model 사용

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=64, activation="relu", input_shape=(32,)))
model.add(tf.keras.layers.Dense(units=64, activation="relu"))
model.add(tf.keras.layers.Dense(units=1, activation="relu"))       

In [ ]:
model.compile(loss="mse", optimizer= "sgd", metrics = "mse")

In [ ]:
model.fit(X_train, y, epochs= 200)

Epoch 1/200
22/22 [==============================] - 2s 2ms/step - loss: 61.2370 - mse: 61.2370
Epoch 2/200
22/22 [==============================] - 0s 2ms/step - loss: 18.1922 - mse: 18.1922
Epoch 3/200
22/22 [==============================] - 0s 2ms/step - loss: 8.6882 - mse: 8.6882
Epoch 4/200
22/22 [==============================] - 0s 2ms/step - loss: 5.6212 - mse: 5.6212
Epoch 5/200
22/22 [==============================] - 0s 2ms/step - loss: 5.1364 - mse: 5.1364
Epoch 6/200
22/22 [==============================] - 0s 2ms/step - loss: 3.0789 - mse: 3.0789
Epoch 7/200
22/22 [==============================] - 0s 2ms/step - loss: 3.5755 - mse: 3.5755
Epoch 8/200
22/22 [==============================] - 0s 2ms/step - loss: 2.6321 - mse: 2.6321
Epoch 9/200
22/22 [==============================] - 0s 2ms/step - loss: 3.3499 - mse: 3.3499
Epoch 10/200
22/22 [==============================] - 0s 2ms/step - loss: 2.7879 - mse: 2.7879
Epoch 11/200
22/22 [==============================] - 0

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
x_test_pred = model.predict(X_test)

12/12 [==============================] - 0s 1ms/step


In [ ]:
r2_score(y_test, x_test_pred)

0.6856171003390445